In [1]:
import tensorflow as tf
import numpy as np
import pathlib
import data

from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.layers              import Dense, LSTM

In [2]:
def getPartialModel(savedModelPath):
    #run the model again to get features -> save features for RNN input
    loadedModel = load_model(savedModelPath)
    
    indexOfGAPLayer = -1
    # Get the index of the last GlobalAveragePooling2D layer starting from the back of the list of layers
    for layer in loadedModel.layers[::-1]:
        layer_type = str(type(layer))
        if layer_type != "<class 'tensorflow.python.keras.saving.saved_model.load.GlobalAveragePooling2D'>":
            indexOfGAPLayer -= 1
        else:
            break

    if indexOfGAPLayer != -1:
        indexAfterGAPLayer = indexOfGAPLayer + 1
        partialModel = Sequential(loadedModel.layers[:indexAfterGAPLayer])
    else:
        # The GlobalAveragePooling2D Layer is the last layer. Thus, we get all the layers (no need to slice the list).
        partialModel = Sequential(loadedModel.layers[:])
    
    return partialModel

In [3]:
def extractFeatures(model, framePath):
    #resizing each frame to 299,299 and getting features
    frame = image.load_img(framePath, 
                           target_size   = (299,299), 
                           interpolation = "lanczos") # shape: (299, 299, 3): # of dim: 3
    frame_arr = image.img_to_array(frame)             # pixel values in range [0,255]
    frame_arr = preprocess_input(frame_arr)           # pixel values in range [-1, 1]
    frame_arr = np.expand_dims(frame_arr, axis = 0)   # expands shape to: (1, 299, 299, 3): # of dim: 4
    features  = model.predict(frame_arr)              # returns numpy array of shape: (1, 2048): # of dim: 2
    features  = features[0]                           # shape: (2048, ): # of dim: 1
    return features

In [4]:
def saveFeatureSequences(model, dataObj):
    # replace [:3] to [:] later, [:3] is here only for testing purposes!
    for dataRow in dataObj.data[:3]:
        sequencePath = pathlib.Path(r"D:\ActionRecognition\Sequences")/dataRow[0]/dataRow[1]/(dataRow[2] + "_featureSequence")
        # get the list of paths of the frames of the video referenced in dataRow
        #framePath is sorted list of frames from specific video
        framePaths = dataObj.getFramesForVideo(dataRow)
        
        featureSequence = []
       
        
        #get features for every frame
        for framePath in framePaths:
            features = extractFeatures(model, framePath)
            featureSequence.append(features)
        return featureSequence

In [5]:
def RNNattempt(featureSequence) :
    #parameters for LTSM 
    timesteps = len(featureSequence) #1 because only 1 frame (timestep=number of frames)
    features = 2048 #size of each feature list
    samples = 1 #because featureSequence = 1 video 
    
    #RNN Model 
    model = Sequential()
    model.add(LSTM(samples, input_shape=(timesteps, features), return_sequences=True))
    model.add(Dropout(0.2)) #prevent overfitting
    model.add(Dense(1, activation='softmax')) #softmax => outputs as probabilities
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  #crossentropy loss function works with softmax act function
     
    #reshape to numpy for model
    data = np.reshape(featureSequence, (samples, timesteps, features))
    print(data.shape)
    
    #Train Model 
    model.fit(data, initial_epoch= 0, epochs=1, batch_size=1)
    model.summary()
    
    
    

In [6]:
def main():
    dataObj        = data.Data()
    #change checkpoint name/version after running trainCNN
    savedModelPath = r"/home/jupyter/action-recognition/Callbacks/CNN/ModelCheckpoint/CNN_002_0.00"
    partialModel   = getPartialModel(savedModelPath)
    saveFeatureSequences(partialModel, dataObj)
    
    #### Attempt to create RNN layer
#     featureSequence = saveFeatureSequences(partialModel, dataObj)
#     RNNattempt(featureSequence)

In [7]:
main()